Preprocessing Actigraphy and Tabular Datasets and merged

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ----------------------------
# 1. Import Necessary Libraries
# ----------------------------
import pandas as pd
import numpy as np
import os
import gc
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    cohen_kappa_score, accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# ----------------------------
# 2. Load and Preprocess Training Data
# ----------------------------

# Function to reduce memory usage
def reduce_memory_usage(df, verbose=True):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        if df[col].dtype in numerics and 'id' not in col:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(df[col].dtype).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Memory usage reduced from {start_mem:.2f} MB to {end_mem:.2f} MB')
    return df

# Load training labels and tabular data
train_csv_path = './dataset/train.csv'
train_labels_df = pd.read_csv(train_csv_path)
print("Training Labels Shape:", train_labels_df.shape)

# Reduce memory usage
train_labels_df = reduce_memory_usage(train_labels_df)

# Extract feature names directly from train_labels_df
all_features = train_labels_df.columns.tolist()
print(f"Number of features in training data: {len(all_features)}")

# Add 'id' and 'sii' to the feature list if not present
if 'id' not in all_features:
    all_features.insert(0, 'id')
if 'sii' not in all_features:
    all_features.append('sii')

# Select available features
train_tabular = train_labels_df[all_features].copy()

# Identify numeric and categorical columns based on data types in train_labels_df
numeric_cols = train_tabular.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_tabular.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ['id', 'sii']]

# Convert 'id' to string if not already
train_tabular['id'] = train_tabular['id'].astype(str)

# Handle missing values in tabular data

# Impute numerical columns with median
num_imputer = SimpleImputer(strategy='median')
train_tabular[numeric_cols] = num_imputer.fit_transform(train_tabular[numeric_cols])

# Impute categorical columns with most frequent
cat_imputer = SimpleImputer(strategy='most_frequent')
train_tabular[categorical_cols] = cat_imputer.fit_transform(train_tabular[categorical_cols])

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    if col in train_tabular.columns:
        le = LabelEncoder()
        train_tabular[col] = le.fit_transform(train_tabular[col])
        label_encoders[col] = le

# Reduce memory usage
train_tabular = reduce_memory_usage(train_tabular)

# ----------------------------
# 3. Extract Actigraphy Features and Merge with Tabular Data
# ----------------------------

missing_actigraphy_ids = []

def extract_actigraphy_features(base_path, ids):
    """
    Extract actigraphy features from partitioned Parquet files for given IDs.
    """
    import pyarrow.parquet as pq

    feature_list = []
    total_ids = len(ids)
    for id_ in tqdm(ids, desc="Extracting Actigraphy Features"):
        id_str = str(id_)
        # Construct the path to the id's parquet files
        id_path = os.path.join(base_path, f'id={id_str}')
        if not os.path.exists(id_path):
            # print(f"Path does not exist for id {id_str}: {id_path}")
            missing_actigraphy_ids.append(id_str)
            continue
        # Read all parquet files under this id directory
        try:
            acti_df = pd.read_parquet(id_path, engine='pyarrow')
            # Add id column if not present
            if 'id' not in acti_df.columns:
                acti_df['id'] = id_str
            else:
                acti_df['id'] = acti_df['id'].astype(str)
            # Compute features
            feature_dict = {'id': id_str}
            # Add all numeric features
            numeric_columns = acti_df.select_dtypes(include=[np.number]).columns.tolist()
            for col in numeric_columns:
                feature_dict[f'{col}_mean'] = acti_df[col].mean()
                feature_dict[f'{col}_std'] = acti_df[col].std()
            feature_list.append(feature_dict)
            del acti_df
            gc.collect()
        except Exception as e:
            print(f"Error reading data for id {id_str}: {e}")
            missing_actigraphy_ids.append(id_str)
            continue
    return pd.DataFrame(feature_list)

# Paths to actigraphy data
actigraphy_train_parquet = './dataset/series_train.parquet/'

# Extract unique IDs from train labels
train_ids = train_tabular['id'].unique().tolist()
print(f"Number of unique training IDs: {len(train_ids)}")

# Extract actigraphy features for training set
train_acti = extract_actigraphy_features(actigraphy_train_parquet, train_ids)
print("Actigraphy Training Features Shape:", train_acti.shape)

# Reduce memory usage
train_acti = reduce_memory_usage(train_acti)

print(f"Unique IDs in train_tabular: {train_tabular['id'].nunique()}")
print(f"Unique IDs in train_acti: {train_acti['id'].nunique()}")
print(f"Duplicated IDs in train_tabular: {train_tabular['id'].duplicated().sum()}")
print(f"Duplicated IDs in train_acti: {train_acti['id'].duplicated().sum()}")

# Merge actigraphy features with tabular data
train_merged = pd.merge(train_tabular, train_acti, on='id', how='inner')
print(train_merged.columns)

# apply preporcessing steps to the merged datased on ACTIGRAPHY data
print(train_merged['non-wear_flag_mean'].value_counts())
# Filter out non-wear periods
if 'non-wear_flag_mean' in train_merged.columns:
    train_merged = train_merged[train_merged['non-wear_flag_mean'] == 0]
    print(f"After filtering non-wear periods, shape: {train_merged.shape}")

# Remove rows where `enmo_mean` is zero
if 'enmo_mean_mean' in train_merged.columns:
    train_merged = train_merged[train_merged['enmo_mean_mean'] > 0]
    print(f"After removing rows with enmo_mean_mean == 0, shape: {train_merged.shape}")

# Replace zeros in `light_mean_mean` with NaN and impute with the column mean
if 'light_mean_mean' in train_merged.columns:
    train_merged['light_mean_mean'] = train_merged['light_mean_mean'].replace(0, np.nan)
    train_merged['light_mean_mean'].fillna(train_merged['light_mean_mean'].mean(), inplace=True)

# Apply log transformation to `light_mean_mean`
if 'light_mean_mean' in train_merged.columns:
    train_merged['log_light_mean_mean'] = np.log1p(train_merged['light_mean_mean'])  # log(1 + x) transformation

merged_num_cols = train_merged.select_dtypes(include=[np.number]).columns.tolist()
if 'sii' in merged_num_cols:
    merged_num_cols.remove('sii')
merged_cat_cols = [col for col in train_merged.columns if col not in merged_num_cols + ['id', 'sii']]

numeric_imputed = num_imputer.fit_transform(train_tabular[numeric_cols])

categorical_imputed = cat_imputer.fit_transform(train_tabular[categorical_cols])

# Impute numerical columns with median
merged_imputer = SimpleImputer(strategy='median')
train_merged[merged_num_cols] = merged_imputer.fit_transform(train_merged[merged_num_cols])

# Reduce memory usage
train_merged = reduce_memory_usage(train_merged)

# ----------------------------
# 7. Save Processed Data to CSV
# ----------------------------

# Save processed training data
processed_train_csv_path = 'processed_train.csv'
train_merged.to_csv(processed_train_csv_path, index=False)
print(f"Processed training data saved to {processed_train_csv_path}")

Training Labels Shape: (3960, 82)
Memory usage reduced from 4.50 MB to 3.42 MB
Number of features in training data: 82
Memory usage reduced from 1.64 MB to 1.34 MB
Number of unique training IDs: 3960


Extracting Actigraphy Features: 100%|██████████| 3960/3960 [00:47<00:00, 83.00it/s] 


Actigraphy Training Features Shape: (996, 27)
Memory usage reduced from 0.19 MB to 0.16 MB
Unique IDs in train_tabular: 3960
Unique IDs in train_acti: 996
Duplicated IDs in train_tabular: 0
Duplicated IDs in train_acti: 0
Merged Training Data Shape: (996, 108)
Merged Training Data Shape: (996, 108)
train_tabular shape: (3960, 82)
train_acti shape: (996, 27)
Merged DataFrame shape: (996, 108)
Columns in train_merged:
Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight',
       ...
       'battery_voltage_mean', 'battery_voltage_std', 'time_of_day_mean',
       'time_of_day_std', 'weekday_mean', 'weekday_std', 'quarter_mean',
       'quarter_std', 'relative_date_PCIAT_mean', 'relative_date_PCIAT_std'],
      dtype='object', length=108)
non-wear_flag_mean
0.000000    348
0.055029      2
0.680572      1
0.984497      1
0.269607      1
          

In [ ]:
# read data
tabular = pd.read_csv('./dataset/train.csv')
tabular.head()

"""## Fixing Errors

### CGAS
The Children's Global Assessment Scale (CGAS) is a rating scale that measures the function of a child. Scores are between 1 and 100 with 1 being the most impaired.
"""
# find rows outside of range and fill with nan
tabular.loc[(tabular['CGAS-CGAS_Score'] > 100) | (tabular['CGAS-CGAS_Score'] < 0),
          'CGAS-CGAS_Score'] = float('nan')

"""### Physical Features"""
# set unrealistic weights and BMIs to NAN
tabular.loc[tabular['Physical-BMI'] < 8, 'Physical-BMI'] = float('nan')

# set unrealistic weights to NAN
tabular.loc[tabular['Physical-Weight'] < 25, 'Physical-Weight'] = float('nan')

"""### Fitness
These are fitness stats from the FitnessGram test.
"""

"""#### Fitness_Endurance-Max_Stage
There are only 22 stages in the endurance test so there is at least two errors. Rows with max stages below 0 or above 22 will be set to NA.
"""

# find rows outside of range and fill with nan
tabular.loc[(tabular['Fitness_Endurance-Max_Stage'] > 22) | (tabular['Fitness_Endurance-Max_Stage'] < 0),
          'Fitness_Endurance-Max_Stage'] = float('nan')

"""#### Fitness_Endurance-Time
Fitness_Endurance-Time is split into 2 columns with the minutes and seconds. I will combine both of these into one column.
"""

# fill na seconds with 0
tabular['Fitness_Endurance-Time_Sec'] =  tabular['Fitness_Endurance-Time_Sec'].fillna(0)

# combine minutes and seconds
tabular['Fitness_Endurance-Time'] = tabular['Fitness_Endurance-Time_Mins'] + tabular['Fitness_Endurance-Time_Sec'] / 60

# drop min and sec columns
tabular.drop(columns=['Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec'], inplace=True)

"""### FitnessGram Child (FGC)"""

"""#### FGC Curlups
You can only get up to 75 curl ups so we can correct those over 75.
"""

# set all curl ups above 75 to 75 and below 0 to nan
tabular.loc[tabular['FGC-FGC_CU'] > 75, 'FGC-FGC_CU'] = 75
tabular.loc[tabular['FGC-FGC_CU'] < 0, 'FGC-FGC_CU'] = float('nan')

# fixing trunk lift and sit and reach values
tabular.loc[tabular['FGC-FGC_TL'] > 12, 'FGC-FGC_TL'] = 12
tabular.loc[tabular['FGC-FGC_TL'] < 0, 'FGC-FGC_TL'] = float('nan')

tabular.loc[tabular['FGC-FGC_SRR'] > 12, 'FGC-FGC_SRR'] = 12
tabular.loc[tabular['FGC-FGC_SRR'] < 0, 'FGC-FGC_SRR'] = float('nan')

tabular.loc[tabular['FGC-FGC_SRL'] > 12, 'FGC-FGC_SRL'] = 12
tabular.loc[tabular['FGC-FGC_SRL'] < 0, 'FGC-FGC_SRL'] = float('nan')

"""### BIA
Bio-electric Impedance Analysis
"""

"""#### Bone Mineral Content

"""

# set extremely unrealistic BMCs to nan tp fix plot
tabular.loc[(tabular['BIA-BIA_BMC'] > 20) | (tabular['BIA-BIA_BMC'] < 0), 'BIA-BIA_BMC'] = float('nan')


"""#### BIA-BIA_BMI
I already did some basic checks for Physical-BMI so I will merge in any BIA-BIA_BMIs into that column where it is missing.
"""

# fillna of Physical BMI with BIA-BIA_BMI
tabular['Physical-BMI'] = tabular['Physical-BMI'].fillna(tabular['BIA-BIA_BMI'])

# drop BIA-BIA_BMI
tabular.drop(columns='BIA-BIA_BMI', inplace=True)

"""#### Basal Metabolic Rate"""

# set extremely unrealistic BMRs to nan to fix plot
tabular.loc[(tabular['BIA-BIA_BMR'] > 5000) | (tabular['BIA-BIA_BMR'] < 500), 'BIA-BIA_BMR'] = float('nan')

"""#### Daily Energy Expenditure"""

# set extremely unrealistic DEEs to nan to fix plot
tabular.loc[(tabular['BIA-BIA_DEE'] > 6000) | (tabular['BIA-BIA_DEE'] < 700), 'BIA-BIA_DEE'] = float('nan')

"""#### Extracellular, Intracellular, and Total Body Water"""

# set extremely unrealistic ECW to nan to fix plot
tabular.loc[tabular['BIA-BIA_ECW'] > 150, 'BIA-BIA_ECW'] = float('nan')

# set extremely unrealistic ICW to nan to fix plot
tabular.loc[tabular['BIA-BIA_ICW'] > 150, 'BIA-BIA_ICW'] = float('nan')

# set extremely unrealistic TBW to nan to fix plot
tabular.loc[tabular['BIA-BIA_TBW'] > 300, 'BIA-BIA_TBW'] = float('nan')


"""#### Fat and Fat Free Mass"""
# set extremely unrealistic FFM to nan to fix plot
tabular.loc[tabular['BIA-BIA_FFM'] > 300, 'BIA-BIA_FFM'] = float('nan')

tabular[tabular['BIA-BIA_FFM'] > .98*tabular['Physical-Weight']][['Physical-Weight', 'BIA-BIA_FFM']]

# set extremely unrealistic FFMI to nan to fix plot
tabular.loc[tabular['BIA-BIA_FFMI'] > 30, 'BIA-BIA_FFMI'] = float('nan')

# serious errors
tabular.loc[tabular['BIA-BIA_FMI'] < 0, 'BIA-BIA_FMI']

# set extremely unrealistic FMI to nan to fix plot
tabular.loc[(tabular['BIA-BIA_FMI'] > 50) | (tabular['BIA-BIA_FMI'] < 0), 'BIA-BIA_FMI'] = float('nan')

# set extremely unrealistic FMI to nan to fix plot
tabular.loc[(tabular['BIA-BIA_Fat'] > 100) | (tabular['BIA-BIA_Fat'] < 0), 'BIA-BIA_Fat'] = float('nan')

# set extremely unrealistic LDM to nan to fix plot
tabular.loc[tabular['BIA-BIA_LDM'] > 100, 'BIA-BIA_LDM'] = float('nan')

# set extremely unrealistic LST to nan to fix plot
tabular.loc[tabular['BIA-BIA_LST'] > 300, 'BIA-BIA_LST'] = float('nan')


# set extremely unrealistic SMM to nan to fix plot
tabular.loc[tabular['BIA-BIA_SMM'] > 150, 'BIA-BIA_SMM'] = float('nan')

"""### Physical Activity Questionnaire
I will combine the two questionnaires into one PAQ_Total column.
"""
# combining reports
tabular['PAQ_Total'] = tabular['PAQ_A-PAQ_A_Total'].fillna(tabular['PAQ_C-PAQ_C_Total'])

tabular.drop(columns=['PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total'], inplace=True)


"""## Save Dataset & Add Actigraphy Data
Save partially cleaned dataset for use in other models
"""

# filter columns
tabular = tabular[['id',  'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height', 'Physical-Weight',
       'Physical-Waist_Circumference','Physical-Diastolic_BP', 'Physical-HeartRate',
        'Physical-Systolic_BP','Fitness_Endurance-Max_Stage','Fitness_Endurance-Time','FGC-FGC_CU',
        'FGC-FGC_CU_Zone', 'FGC-FGC_GSND','FGC-FGC_GSND_Zone', 'FGC-FGC_GSD',
        'FGC-FGC_GSD_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL',
        'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR','FGC-FGC_SRR_Zone', 'FGC-FGC_TL',
         'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_Total', 'SDS-SDS_Total_Raw','SDS-SDS_Total_T',
       'PreInt_EduHx-computerinternet_hoursday', 'sii', 'PCIAT-PCIAT_Total']]
# read in
actigraphy = pd.read_csv('processed_train.csv')
actigraphy = actigraphy[['id', 'step_mean', 'step_std', 'X_mean', 'X_std', 'Y_mean', 'Y_std', 'Z_mean',
       'Z_std', 'enmo_mean', 'enmo_std', 'anglez_mean', 'anglez_std',
       'non-wear_flag_mean', 'non-wear_flag_std', 'light_mean', 'light_std',
       'battery_voltage_mean', 'battery_voltage_std', 'time_of_day_mean',
       'time_of_day_std', 'weekday_mean', 'weekday_std', 'quarter_mean',
       'quarter_std', 'relative_date_PCIAT_mean', 'relative_date_PCIAT_std']]


print("Tabular Columns:", tabular.columns)
print("Actigraphy Columns:", actigraphy.columns)


# left merge tabular and actigraphy on id
tabular = pd.merge(tabular, actigraphy, on='id', how='left')
tabular

tabular.to_csv('train_cleaned_actigraphy.csv', index=False)

Tabular Columns: Index(['id', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score',
       'Physical-BMI', 'Physical-Height', 'Physical-Weight',
       'Physical-Waist_Circumference', 'Physical-Diastolic_BP',
       'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time', 'FGC-FGC_CU',
       'FGC-FGC_CU_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD',
       'FGC-FGC_GSD_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL',
       'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL',
       'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_Total', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
       'PreInt_EduHx-computerinternet_hoursday', 'sii', 'PCIAT-PCIAT_Total